In [ ]:
import time
import os
import math
import numpy as np
from pydrake.all import (
    InverseKinematics,
    AddMultibodyPlantSceneGraph,
    BsplineTrajectory,
    DiagramBuilder,
    KinematicTrajectoryOptimization,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MinimumDistanceLowerBoundConstraint,
    Parser,
    PositionConstraint,
    Rgba,
    ConstantVectorSource,
    RigidTransform,
    Role,
    Solve,
    Sphere,
    StartMeshcat,
    TrajectorySource,
    Simulator,
    RotationMatrix,
    PiecewisePose,
    GenerateHtml
)

import pydot
from IPython.display import HTML, SVG, display

from pydrake.common import temp_directory
from pydrake.trajectories import PiecewisePolynomial
from manipulation.station import AddPointClouds, MakeHardwareStation, load_scenario
from manipulation import running_as_notebook
from manipulation.meshcat_utils import PublishPositionTrajectory
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddShape, AddWsg
from manipulation.utils import ConfigureParser

In [ ]:
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://118e769b-4a3a-40aa-8d8d-69e00b8c0bfd.deepnoteproject.com/7002/


In [ ]:
x = np.random.rand() * 0.3 + 0.3
y = np.random.rand() * 0.4 - 0.2
theta = np.random.rand() * 180

In [ ]:
scenario_data = """
directives:
- add_model:
    name: table_top
    file: file:///work/table.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: box
    file: file:///work/box.sdf
    default_free_body_pose:
        base_link_cracker:
            translation: [""" + str(x) + ", " + str(y) +""", 0.1]
            rotation: !Rpy { deg: [-90, 0, """ + str(theta) + """] }
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0]
        iiwa_joint_3: [0]
        iiwa_joint_4: [0]
        iiwa_joint_5: [0]
        iiwa_joint_6: [0]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {deg: [90, 0, 90]}
- add_model:
    name: camera
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: world
    child: camera::base
    X_PC:
        translation: [0, 0, 3]
        # Point slightly down towards camera
        # RollPitchYaw(0, -0.2, 0.2) @ RollPitchYaw(-np.pi/2, 0, np.pi/2)
        rotation: !Rpy { deg: [180, 0, 0] }
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
cameras:
    main_camera:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera::base
"""

scenario = load_scenario(data=scenario_data)

In [ ]:
with open("/work/box.sdf", "r") as box_sdf_file:
    box_sdf = box_sdf_file.read()

with open("/work/table.sdf", "r") as table_sdf_file:
    table_sdf = table_sdf_file.read()

In [ ]:
def get_trajectory(x, y, height, theta):
    def create_q_knots(pose_lst):
        """Convert end-effector pose list to joint position list using series of
        InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
        contain gripper joints, but these should not matter to the constraints.
        @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
        @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
        """

        controller_builder = DiagramBuilder()
        plant, _ = AddMultibodyPlantSceneGraph(controller_builder, time_step=0.0001)
        controller_iiwa = AddIiwa(plant)
        controller_wsg = AddWsg(plant, controller_iiwa, roll=0.0, welded=True, sphere=True)

        plant.Finalize()

        q_knots = []
        world_frame = plant.world_frame()
        gripper_frame = plant.GetFrameByName("body")
        q_nominal = np.array(
            [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0]
        )  # nominal joint angles for joint-centering.

        def AddOrientationConstraint(ik, R_WG, bounds):
            """Add orientation constraint to the ik problem. Implements an inequality
            constraint where the axis-angle difference between f_R(q) and R_WG must be
            within bounds. Can be translated to:
            ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
            """
            ik.AddOrientationConstraint(
                frameAbar=world_frame,
                R_AbarA=R_WG,
                frameBbar=gripper_frame,
                R_BbarB=RotationMatrix(),
                theta_bound=bounds,
            )

        def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
            """Add position constraint to the ik problem. Implements an inequality
            constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
            translated to
            ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
            """
            ik.AddPositionConstraint(
                frameA=world_frame,
                frameB=gripper_frame,
                p_BQ=np.zeros(3),
                p_AQ_lower=p_WG_lower,
                p_AQ_upper=p_WG_upper,
            )

        initial_guess = q_nominal

        for i in range(len(pose_lst)):
            ik = inverse_kinematics.InverseKinematics(plant)
            q_variables = ik.q()  # Get variables for MathematicalProgram
            prog = ik.prog()  # Get MathematicalProgram

            d = 0.02

            AddOrientationConstraint(ik, pose_lst[i].rotation(), 0)

            AddPositionConstraint(ik, pose_lst[i].translation() - [d, 0, d], pose_lst[i].translation() + [d, 0, d])

            prog.AddCost((q_variables - q_nominal).dot(q_variables - q_nominal))

            prog.SetInitialGuess(ik.q(), initial_guess)

            result = Solve(prog)

            print (i)
            assert result.is_success()

            initial_guess = result.GetSolution(q_variables)

            q_knots.append(result.GetSolution(q_variables))

        return q_knots
    
    import math

    w = 0.158000
    h = height

    theta_0 = math.atan(w / h)

    lower_dtheta = 3 * np.pi / 2 - theta_0
    upper_dtheta = 3 * np.pi / 2 - theta_0 / 2

    delta_h = 0.49 + w / 2 - math.sqrt(w**2 + h**2) / 2

    lower_omega = lower_dtheta * math.sqrt(9.8067 * 0.5 / delta_h)
    upper_omega = upper_dtheta * math.sqrt(9.8067 * 0.5 / delta_h)

    eps = 0.002
    def create_trajectory(x, y, end_angle, end_rot_velocity):
        end_rot_velocity *= 0.97
        def create_pose_lst(time_to_throw):

            new_time = time_to_throw + eps
            new_omega = end_rot_velocity + eps * end_rot_velocity / time_to_throw
            new_end_angle = new_time * new_omega / 2

            times = [0, 0.5, 2, 3, 5.5, 7.9, 8, 8 + new_time, 8 + new_time + 0.1]

            r = RotationMatrix.MakeXRotation(-np.pi / 2)

            small_rot = RigidTransform(RotationMatrix.MakeZRotation(np.pi / 8.0))
            x_rot = RigidTransform(RotationMatrix.MakeZRotation(new_end_angle))

            y_rot = RigidTransform(RotationMatrix.MakeYRotation(np.pi - theta))
            #y_rot = RotationMatrix.MakeYRotation(theta * np.pi / 180)
            z_rot = RigidTransform(RotationMatrix.MakeZRotation(-np.pi / 2))

            rot = RotationMatrix([
                [-1, 0, 0],
                [0, 0, -1],
                [0, -1, 0],
            ])

            print ("Angle:")
            print (theta)
            print (np.sin(theta))
            print (np.cos(theta))

            frames = [RigidTransform(rot, [x, y, height + 0.3] - 0.2 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])) @ y_rot, 
            RigidTransform(rot, [x, y, height + 0.2] - 0.2 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])) @ y_rot, 
            RigidTransform(rot, [x, y, height - 0.01] - 0.2 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])) @ y_rot,
            RigidTransform(rot, [x, y, height + 0.01] - 0.2 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])) @ y_rot,
            RigidTransform(rot, [x, y, height + 0.4] - 0.2 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])) @ y_rot,
            RigidTransform(rot, [0.65, 0, 0.55]), RigidTransform(rot, [0.65, 0, 0.55]),
            RigidTransform(rot, [0.65, 0, 0.55]) @ x_rot, RigidTransform(rot, [0.35, 0, 0.65])]
            '''
            frames = [RigidTransform(rot @ y_rot, [x, y, height + 0.2] + 0.3 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])), 
            RigidTransform(rot @ y_rot, [x, y, height + 0.1] + 0.3 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])), 
            RigidTransform(rot @ y_rot, [x, y, height - 0.01] + 0.3 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])),
            RigidTransform(rot @ y_rot, [x, y, height + 0.01] + 0.3 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])),
            RigidTransform(rot @ y_rot, [x, y, height + 0.4] + 0.3 * np.array([0.1 * np.cos(theta), 0.1 * np.sin(theta), 0])),
            RigidTransform(rot, [0.65, 0, 0.55]), RigidTransform(rot, [0.65, 0, 0.55]),
            RigidTransform(rot, [0.65, 0, 0.55]) @ x_rot, RigidTransform(rot, [0.35, 0, 0.65])]'''

            return times, frames
        
        time_to_throw = 2 * end_angle / end_rot_velocity
        times, frames = create_pose_lst(time_to_throw)

        q_knots = np.array(create_q_knots(frames))
        q_traj = PiecewisePolynomial.CubicShapePreserving(times, q_knots[:, 0:7].T)

        g_knots = np.array([0.1, 0.1, 0, 0, 0.1, 0.1])
        gripper_knots = g_knots.reshape(1, g_knots.shape[0])

        gripper_t_lst = [0, 2.2, 2.6, 8 + time_to_throw - 0.001, 8 + time_to_throw, 12]
        g_traj = PiecewisePolynomial.ZeroOrderHold(gripper_t_lst, gripper_knots)

        return q_traj, g_traj, frames
    
    return create_trajectory(x, y, np.pi / 2, lower_omega)

In [ ]:
# Perception

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0004)
station = builder.AddSystem(
    MakeHardwareStation(scenario, meshcat=meshcat)
)

iiwa = AddIiwa(plant)
wsg = AddWsg(plant, iiwa, roll=0.0, welded=True, sphere=True)

parser = Parser(plant)
ConfigureParser(parser)

table = parser.AddModelsFromString(table_sdf, "sdf")[0]
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("table_top_link"),
    RigidTransform([0, 0, -0.5]),
)

box = parser.AddModelsFromString(box_sdf, "sdf")[0]

plant.Finalize()

# Export the camera outputs
builder.ExportOutput(
    station.GetOutputPort("camera0.rgb_image"), "rgb_image"
)
builder.ExportOutput(
    station.GetOutputPort("camera0.depth_image"), "depth_image"
)

to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)

# Add a box for the camera in the environment.
plant = station.GetSubsystemByName("plant")
camera_instance = plant.GetModelInstanceByName("camera")

scene_graph = station.GetSubsystemByName("scene_graph")
AddMultibodyTriad(
    plant.GetFrameByName("base", camera_instance),
    scene_graph,
    length=0.1,
    radius=0.005,
)

# Export the point cloud output.
builder.ExportOutput(
    to_point_cloud['camera0'].point_cloud_output_port(), "point_cloud"
)

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)

pc = diagram.GetOutputPort("point_cloud").Eval(diagram_context)

"""
meshcat.SetObject(f"pointcloud", pc, point_size=0.005)

meshcat.SetObject("pick", Sphere(0.02), rgba=Rgba(0.1, 0.1, 0.9, 1))
meshcat.SetTransform("pick", RigidTransform([0.1, 0, 0]))

meshcat.SetObject("pick2", Sphere(0.02), rgba=Rgba(0.1, 0.1, 0.9, 1))
meshcat.SetTransform("pick2", RigidTransform([0, 0.15, 0]))
"""

all_pc_xyz = pc.xyzs()

include = np.logical_and(np.logical_or(np.abs(all_pc_xyz[0, :]) > 0.2, np.abs(all_pc_xyz[1, :]) > 0.2), all_pc_xyz[2, :] > 0.05)
#include = np.logical_and(np.abs(all_pc_xyz[0, :]) > 0.05, np.abs(all_pc_xyz[1, :]) > 0.06)
without_arm = all_pc_xyz[:, include]

top_z = np.max(without_arm[2, :])

obj_xys = without_arm[:2, :]

min_x_point = obj_xys[:, np.argmin(obj_xys[0, :])]
max_x_point = obj_xys[:, np.argmax(obj_xys[0, :])]
min_y_point = obj_xys[:, np.argmin(obj_xys[1, :])]
max_y_point = obj_xys[:, np.argmax(obj_xys[1, :])]

center_x = (min_x_point[0] + max_x_point[0]) / 2.0
center_y = (min_y_point[1] + max_y_point[1]) / 2.0

pt1 = min_y_point

if np.linalg.norm(min_x_point - min_y_point) > np.linalg.norm(max_x_point - min_y_point):
    pt2 = min_x_point
else:
    pt2 = max_x_point

slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0])

theta_x = np.arctan(slope)

meshcat.Delete()

print (top_z, center_x, center_y, theta_x)

0.20720959 0.40372633934020996 -0.4329555034637451 0.35995513


In [ ]:
q_traj, g_traj, frames = get_trajectory(center_x, center_y, top_z, theta_x)

Angle:
0.35995513
0.35223225
0.9359126
0
1
2
3
4
5
6
7
8


In [ ]:
#GREEN
#meshcat.SetObject("abovepick", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
#meshcat.SetTransform("abovepick", frames[1])

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0004)
station = builder.AddSystem(
    MakeHardwareStation(scenario, meshcat=meshcat)
)

iiwa = AddIiwa(plant)
wsg = AddWsg(plant, iiwa, roll=0.0, welded=True, sphere=True)

parser = Parser(plant)
ConfigureParser(parser)

table = parser.AddModelsFromString(table_sdf, "sdf")[0]
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("table_top_link"),
    RigidTransform([0, 0, -0.5]),
)

box = parser.AddModelsFromString(box_sdf, "sdf")[0]

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))

g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

builder.Connect(
    q_traj_system.get_output_port(), station.GetInputPort("iiwa.position")
)

builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
)

for i in range (1, 8):
    plant.GetJointByName("iiwa_joint_" + str(i)).set_velocity_limits(
        [-np.inf], [np.inf]
    )
    plant.GetJointByName("iiwa_joint_" + str(i)).set_position_limits(
        [-np.inf], [np.inf]
    )
    
plant.Finalize()

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

simulator = Simulator(diagram, context)

diagram.SetDefaultContext(context)
plant_context = plant.GetMyContextFromRoot(root_context=simulator.get_context())

#q_traj_system.UpdateTrajectory(q_traj)
#g_traj_system.UpdateTrajectory(g_traj)

meshcat.StartRecording(set_visualizations_while_recording=False)
simulator.AdvanceTo(10)

meshcat.PublishRecording()

In [ ]:
#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=118e769b-4a3a-40aa-8d8d-69e00b8c0bfd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>